<a href="https://colab.research.google.com/github/sheldonkemper/portfolio/blob/main/CAM_DS_301_Sentiment_analysis_and_text_classification_Activity_2_2_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Activity 2.2.6 Sentiment analysis and text classification

In this activity, you will build a sentiment analysis model using Python and a data set of customer reviews. You will preprocess the data and fine-tune, evaluate, and test the model.


## Objective
In this activity, you will download a data set from Hugging Face and conduct text classification on it. Your objective is to analyse how different parameter choices affect the performance of a sentiment classifier.

You will complete this in your Notebook, where you will:

- create and train sentiment classifier RNN models
- evaluate model performance.



## Assessment criteria

By completing this activity, you will be able to provide evidence that you can:
*   apply various text preprocessing techniques and representation methods to preprocess and analyse textual data.
*   comprehend and implement different types of recurrent neural networks (RNNs) and understand their applications in NLP.
*   build and fine-tune advanced NLP models for specific natural language processing tasks.


## Activity guidance

1. Install the necessary packages that will be useful in this activity
2. Load the dataset sst5 from hugging face (https://huggingface.co/datasets/SetFit/sst5)

3. Create dataframes of the train and train split
4. Split the train dataframe into train and validation in the ratio of 8:2
5. Preprocess the dataset, set the maximum size to 200, vocabulary size to 30000
6. During tokenisation, mark out of vocabulary words as "[OOV]"
7. Pad your sequences with special tokens
8. Train a sentiment classifier on the dataset and compare different models for text classification
9.Train for 5 epochs
- Train with a vanilla RNN
- Train with an LSTM
- Is there any difference between a GRU and an LSTM?
- Train with a bidirectional LSTM
10. Comment on the performance of all the models


> Start your activity here. Select the pen from the toolbar to add your entry.

In [ ]:
#In this activity, you will be required to download a dataset from huggingface and perfom the text classification on the the dataset
#You will be required to study the impact of different different parameter choices on the classification perfomance of sentiment classifier


#1. Install the necessary packages that will be useful in this  activity
#2. Load the dataset sst5 from hugginface (https://huggingface.co/datasets/SetFit/sst5)
#3.Create dataframes of the train and train split
#4 Split the train dataframe into train and validation in the ratio of 8:2
#5 Preprocess the dataset,  set the maximum size to 200, vocabulary size to 30000
#6. During tokenization, mark out of vocabulary words as "<OOV>"
#7 Pad your sequences with special tokens
#8. Train a sentiment classifier on the dataset and compare different models for text classification
# Train for 5 epochs
#    - Train with a vanilla RNN
#    - Train with an LSTM
#    - Is there any difference between a GRU and an LSTM?
#    - Train with a bidirectional LSTM
# Comment on the perfomance of all the models